In [1]:
import numpy as np

In [3]:
class MultinomialNBCustom:
    def __init__(self):
        self.X = None
        self.y = None
    
    def _calculate_probability(self, classes):
        self.p_fs = {}
        for cls in classes:
            N = self.X[self.y == cls].sum()
            self.p_fs[cls] = []
            for feat in range(self.X.shape[1]):
                self.p_fs[cls].append(self.X[self.y == cls][:, feat].sum()/N)
        self.priors = {}
        for cls in classes:
            self.priors[cls] = (self.y == cls).sum()/len(self.y)
        
        return self
    
    def _calculate_likelihood(self, X, cls):
        N = self.X[self.y == cls].sum()
        likelihood = np.math.factorial(N)
        for ix in range(len(X)):
            likelihood *= (np.math.pow(self.p_fs[cls][ix], X[ix])/np.math.factorial(X[ix]))
        
        return likelihood

    def _calculate_unnormalized_posterior(self, X, cls):
        return self.priors[cls]*self._calculate_likelihood(X, cls)

    def fit(self, X, y):
        self.X = X
        self.y = y

        self._calculate_probability(np.unique(y))
        return self
    
    def predict(self, X):
        assert len(X.shape) == 2, "Shape of X passed is not correct"
        y_pred = []
        for x in X:
            posteriors_probs = {}
            for cls in self.priors.keys():
                posteriors_probs[cls] = self._calculate_unnormalized_posterior(self, x, cls)
            for key in posteriors_probs.keys():
                posteriors_probs[key] /= np.sum(posteriors_probs.values())
            
            sorted_probs = sorted(posteriors_probs.keys(), key=lambda x: posteriors_probs[key], reverse=True)
            y_pred.append({"class": sorted_probs[0], "confidence":posteriors_probs[sorted_probs[0]]})

        return y_pred
